<a href="https://colab.research.google.com/github/Edriczz/Formal_logic/blob/main/Formal_Logic_EAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install z3-solver

In [3]:
from z3 import *

# --- Problem Setup ---
# Define grid boundaries and key coordinates from the exam document.
START_POS = (0, 0)
END_POS = (6, 4)

# Interpretation of the red (forbidden) coordinates from Figure 1 .
# A robot cannot pass through these points.
RED_DOTS = [(2, 1), (4, 1), (2, 2), (3, 2), (4, 2), (2, 3), (3, 3)]

# Interpretation of coordinates to be checked based on labels in Figure 1.
# Note the contradictions with RED_DOTS for C and D.
POS_C = (2, 1)
POS_D = (3, 2)
POS_E = (4, 3)

def solve_and_count_all_paths(solver):
    """
    Finds and counts all unique solutions (paths) for a given solver setup.
    It iteratively finds a model and adds a constraint to exclude it in the
    next search, until no more models can be found.
    """
    count = 0
    while solver.check() == sat:
        count += 1
        model = solver.model()
        # Exclude the current solution to find the next one
        path_block = []
        for var in model.decls():
            path_block.append(var() != model[var])
        solver.add(Or(path_block))
    return count

# --- Question 1: Module (a) ---
print("--- Question 1: Module (a) Analysis ---")

# The path length for module (a) is fixed. To get from (0,0) to (6,4)
# with only unit horizontal/vertical moves, the robot must make
# exactly 6 steps right and 4 steps up.
# Total steps = 6 + 4 = 10. So, we model a path of length 11 (0 to 10).
PATH_LEN_A = 10

# 1. Create SMT encoding for module (a) movements .
x = [Int(f"x_{i}") for i in range(PATH_LEN_A + 1)]
y = [Int(f"y_{i}") for i in range(PATH_LEN_A + 1)]

# Create a solver for Question 1
s1 = Solver()

# Add constraints for initial and final coordinates .
s1.add(x[0] == START_POS[0], y[0] == START_POS[1])
s1.add(x[PATH_LEN_A] == END_POS[0], y[PATH_LEN_A] == END_POS[1])

# Add constraints for robot movement rules for each step .
for i in range(PATH_LEN_A):
    # Robot can only move one step vertically or horizontally
    step_constraint = Or(
        And(x[i+1] == x[i] + 1, y[i+1] == y[i]),  # Move right
        And(x[i+1] == x[i], y[i+1] == y[i] + 1)   # Move up
    )
    s1.add(step_constraint)

# Add constraints to avoid all red (forbidden) coordinates .
for i in range(1, PATH_LEN_A): # Check intermediate points
    for red_x, red_y in RED_DOTS:
        s1.add(Not(And(x[i] == red_x, y[i] == red_y)))

# 1(i). Find the number of all possible robot's movements (path) from A to B .
num_paths_a = solve_and_count_all_paths(s1)
print(f"1(i): Found {num_paths_a} possible paths for module (a).\n")

# 1(ii). Show that there is no robot movement that passes coordinate C .
# We test this by creating a new solver with an additional constraint that
# the path *must* pass through C. If it's UNSAT, the proposition is true.
s1_check_c = Solver()
# Add all the same constraints as before
s1_check_c.add(s1.assertions())

# Add the new constraint: the path must pass through C(2,1) at some step.
path_must_pass_c = Or([And(x[i] == POS_C[0], y[i] == POS_C[1]) for i in range(1, PATH_LEN_A)])
s1_check_c.add(path_must_pass_c)

print("1(ii): Checking if any path can pass through coordinate C(2,1)...")
if s1_check_c.check() == unsat:
    print("Result: UNSAT. This proves there is no movement that passes through coordinate C.")
    print("Reason: Coordinate C(2,1) is a forbidden 'red dot'.\n")
else:
    print("Result: SAT. This would mean a path can pass through C, which contradicts the problem statement.\n")




--- Question 1: Module (a) Analysis ---
1(i): Found 10 possible paths for module (a).

1(ii): Checking if any path can pass through coordinate C(2,1)...
Result: UNSAT. This proves there is no movement that passes through coordinate C.
Reason: Coordinate C(2,1) is a forbidden 'red dot'.



In [4]:
# --- Question 2: Module (b) ---
print("--- Question 2: Module (b) Analysis ---")

def create_module_b_solver(path_len):
    """Helper function to create a Z3 solver for a given path length for module (b)."""
    s = Solver()

    # 2. Create a SMT encoding to define the movements for module (b) .
    x_b = [Int(f"xb_{i}") for i in range(path_len + 1)]
    y_b = [Int(f"yb_{i}") for i in range(path_len + 1)]

    # Initial and final state constraints
    s.add(x_b[0] == START_POS[0], y_b[0] == START_POS[1])
    s.add(x_b[path_len] == END_POS[0], y_b[path_len] == END_POS[1])

    # Movement constraints for module (b) .
    for i in range(path_len):
        s.add(Or(
            And(x_b[i+1] == x_b[i] + 1, y_b[i+1] == y_b[i]),      # Horizontal
            And(x_b[i+1] == x_b[i], y_b[i+1] == y_b[i] + 1),      # Vertical
            And(x_b[i+1] == x_b[i] + 1, y_b[i+1] == y_b[i] + 1)   # Up-diagonal
        ))

    # Forbidden zone constraints
    for i in range(1, path_len):
        for red_x, red_y in RED_DOTS:
            s.add(Not(And(x_b[i] == red_x, y_b[i] == red_y)))

    return s, x_b, y_b

# For module (b), path length is variable. Min length is 6, max is 10.
MIN_PATH_LEN_B = 6
MAX_PATH_LEN_B = 10

# 2(i). Find the number of all possible robot's movements (path) from A to B .
total_paths_b = 0
path_counts_by_length = {}
print("2(i): Finding all possible paths for module (b) by checking path lengths 6 to 10...")
for k in range(MIN_PATH_LEN_B, MAX_PATH_LEN_B + 1):
    s_b, _, _ = create_module_b_solver(k)
    num_paths = solve_and_count_all_paths(s_b)
    if num_paths > 0:
        path_counts_by_length[k] = num_paths
        total_paths_b += num_paths
    print(f"  - Paths of length {k}: {num_paths}")
print(f"Result: Found a total of {total_paths_b} possible paths for module (b).\n")


# 2(ii). Find the number of all possible robot's minimal movements (path) from A to B .
print("2(ii): Finding the number of minimal movements...")
minimal_length = -1
minimal_path_count = 0
for length, count in sorted(path_counts_by_length.items()):
    if count > 0:
        minimal_length = length
        minimal_path_count = count
        break

if minimal_length != -1:
    print(f"Result: The minimal path length is {minimal_length}. There are {minimal_path_count} such paths.\n")
else:
    print("Result: No paths were found for module (b).\n")

# 2(iii). Show that all minimal movements must pass coordinates C, D and E .
print("2(iii): Checking if all minimal paths must pass through C, D, and E.")
print("         NOTE: This is impossible as written, since C(2,1) and D(3,2) are red dots.")

if minimal_length != -1:
    # We will disprove the statement by checking each point.
    # To prove a path MUST pass a point, we show that AVOIDING the point is UNSAT.

    points_to_check = {'C': POS_C, 'D': POS_D, 'E': POS_E}
    for name, pos in points_to_check.items():
        s_check, x_vars, y_vars = create_module_b_solver(minimal_length)

        # Add constraint to AVOID the point
        path_avoids_point = And([Not(And(x_vars[i] == pos[0], y_vars[i] == pos[1])) for i in range(1, minimal_length)])
        s_check.add(path_avoids_point)

        print(f"  - Checking point {name}{pos}: If we force paths to AVOID this point...")
        if s_check.check() == unsat:
            # This means no path exists that avoids the point, so all paths must pass it.
            print(f"    Result: UNSAT. All minimal paths MUST pass through {name}{pos}.")
        else:
            # This means at least one path exists that avoids the point.
            num_avoiding = solve_and_count_all_paths(s_check)
            print(f"    Result: SAT. Found {num_avoiding} minimal path(s) that AVOID {name}{pos}.")
            print(f"    Conclusion: The statement that all paths must pass through {name} is FALSE.")

--- Question 2: Module (b) Analysis ---
2(i): Finding all possible paths for module (b) by checking path lengths 6 to 10...
  - Paths of length 6: 0
  - Paths of length 7: 0
  - Paths of length 8: 6
  - Paths of length 9: 16
  - Paths of length 10: 10
Result: Found a total of 32 possible paths for module (b).

2(ii): Finding the number of minimal movements...
Result: The minimal path length is 8. There are 6 such paths.

2(iii): Checking if all minimal paths must pass through C, D, and E.
         NOTE: This is impossible as written, since C(2,1) and D(3,2) are red dots.
  - Checking point C(2, 1): If we force paths to AVOID this point...
    Result: SAT. Found 6 minimal path(s) that AVOID C(2, 1).
    Conclusion: The statement that all paths must pass through C is FALSE.
  - Checking point D(3, 2): If we force paths to AVOID this point...
    Result: SAT. Found 6 minimal path(s) that AVOID D(3, 2).
    Conclusion: The statement that all paths must pass through D is FALSE.
  - Checking